# Felix Baumgartners Stratosphärensprung

## (Numerische Lösung von DGL höherer Ordnung)

Am 14. Oktober 2012 führte Felix Baumgartner einen Fallschirmsprung aus 39 km Höhe aus (Stratosphärensprung).

<center><img src="figs/Sprung_Baumgartner_tile_x1.jpg" width=1000 /> </center>

Fakten aus der Presse:
- Absprunghöhe: 39 km
- Maximalgeschwindigkeit (Überschall): 1358 km/h (377.2 m/s) nach ca. 50 Sekunden
- Freier Fall: 4 Min. und 20 Sekunden
- Fallschirm geöffnet bei ca. 2000 m über Grund
- Gesamtdauer des Sprungs: ca. 10 Min.

## Modellierung des Sprungs (freier Fall mit Reibung)

$$
m\ddot{z}(t)=-mg+\underbrace{\frac 12 \rho(z)C_w(z)A(z)\dot{z}(t)^2}_{F_R(z, t)}
$$

Die DGL muss für zwei Phasen des Sprungs separat gelöst werden!

<img src="figs/skydive_sketch.png" style="margin:25px;" align="left" width=300 />

<br />
<br />
<br />

- $g$: &nbsp; &nbsp; &nbsp; &nbsp; Schwerekraftbeschleunigung (9.8 ms$^{-2}$).
- $m$: &nbsp; &nbsp; &nbsp; &nbsp; Masse von Felix Baumgartner inkl. Ausrüstung (118 kg).
- $\rho(z)$: &nbsp; &nbsp; Dichte der Luft
- $A(z)$: &nbsp; &nbsp; Wirkungsquerschnitt des Springers
- $C_w(z)$: &nbsp; Widerstandsbeiwert (Formfaktor)

## Konstanten und Anfangsbedingungen:

<img src="figs/Sprung_2_final.jpg" style="margin:25px;" align="left" width=200 />

  <br />
  
  - $t_0 = 0$ s, $z(t_0) = 39$ km; $\dot{z}(t_0)=0$ m/s
  - $g=9.8$ ms$^{-2}$, $m=118$ kg, $A(z) =0.9$ m$^2$, $C_w(z)=0.3$
  - $\rho(z) = 1.22\exp(-\frac z{10\text{ km}})$ kg m$^{-3}$

<img src="figs/Sprung_3_final.jpg" style="margin:25px;" align="left" width=200 />

  <br />
  
  - $t_f =$ ?, $z(t_f)\approx 2.0$ km, $\dot{z}(t_f)=$ ? ($t_f$ und $\dot{z}(t_f)$ ergeben sich aus Phase 1)
  - $g=9.8$ ms$^{-2}$, $m=118$ kg, $A(z)=50$ m$^2$, $C_w(z)=1.5$
  - $\rho(z)=1.22$ kg m$^{-3}$

# Lösung in Python

### Überführung einer DGL 2. Ordnung in ein System aus 2 gekoppelten DGL 1. Ordnung

Allgemeine DGL 2. Ordnung: 
\begin{equation}
\ddot{x}(t) = f(x(t), \dot{x}(t), t) \tag{1}
\end{equation}

Definiere $w(t) = \dot{x}(t)$ und wir erhalten für (1) das DGL-System:
$$
\begin{eqnarray}
\dot{x}(t) & = & w(t) \\
\dot{w}(t) & = & f(x(t), w(t), t)
\end{eqnarray}
$$

**Beachte:** Zur numerischen Lösung werden Anfangsbedingungen für $x(t_0)$ und $w(t_0)=\dot{x}(t_0)$ benötigt. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Luftdichte für Phase 1:
def rho_phase1(z):
    # Wir rechnen in Einheiten von Metern:
    return 1.22 * np.exp(-z / 10000)

1. Bringe die zweite Ableitung auf die linke Seite der Gleichung und den Rest auf die rechte
$$
\ddot{z}(t)=-g+\frac 1{2m} \rho(z)C_w(z)A(z)\dot{z}(t)^2
$$

2. Transformiere die DGL in ein System aus gekoppelten DGL 1. Ordnung

$$
\begin{eqnarray}
\dot{z}(t) & = & w(t) \\
\dot{w}(t) & = & -g+\frac 1{2m} \rho(z)C_w(z)A(z)w(t)^2
\end{eqnarray}
$$

3. Definiere $\vec{S}=(z, w)$ und bringe das DGL-System in Vektorform:
$$ \vec{S} = \begin{bmatrix} z(t) \\ w(t) \end{bmatrix} \hspace{5mm} \implies  \hspace{5mm} \frac{\rm{d}\vec{S}}{\rm{d}t} = \begin{bmatrix} \dot{z}(t) \\ \dot{w}(t) \end{bmatrix} = \begin{bmatrix} w(t) \\ -g + \frac 1{2m}\rho(z)C_w(z)A(z)w(t)^2 \end{bmatrix}$$

3. definiere eine Python-Funktion mit Argumenten (S, t, ...) und Rückgabewert ${\rm d}\vec{S}/{\rm d}t$: 

In [ ]:
## SOLUTION
def dSdt(S, t, g, m, phase):
    z, w = S
    
    if phase == 1:
        A = 0.9
        C_w = 0.3
        rho = rho_phase1(z)
    else:
        A = 50
        C_w = 1.5
        rho = 1.22
        
    return [ w, 
             -g + rho * C_w * A * w**2 / (2. * m) ]    

4. definiere Zeiten, an denen das DGL-System gelöst werden soll, die Anfangsbedingung und nötige Konstanten. Zuletzt mit allem die DGL-Routine `odeint` aufrufen:

In [ ]:
## SOLUTION

# phase 1:
t_phase1 = np.linspace(0.0, 300, 1000)
g = 9.81
m = 118
phase = 1
w0 = 0
z0 = 39000

S_phase1 = odeint(dSdt, t=t_phase1, y0=(z0, w0), args=(g, m, phase))
z_phase1 = S_phase1[:,0]
w_phase1 = S_phase1[:,1]

# alternativ:
#S_phase1 = odeint(dSdt, t=t_phase1, y0=(z0, w0), args=(g, m, phase)).T
#z_phase1, w_phase1 = S_phase1

In [ ]:
## SOLUTION

mask = z_phase1 > 2000
z_phase1 = z_phase1[mask]
w_phase1 = w_phase1[mask]
t_phase1 = t_phase1[mask]

In [ ]:
## SOLUTION

t_phase2 = np.linspace(t_phase1[-1], 700, 1000)
g = 9.81
m = 118
phase = 2
w0 = w_phase1[-1]
z0 = z_phase1[-1]

S_phase2 = odeint(dSdt, t=t_phase2, y0=(z0, w0), args=(g, m, phase))
z_phase2 = S_phase2[:,0]
w_phase2 = S_phase2[:,1]

# alternativ:
#S_phase2 = odeint(dSdt, t=t_phase2, y0=(z0, w0), args=(g, m, phase)).T
#z_phase2, w_phase2 = S_phase2

In [ ]:
## SOLUTION

mask = z_phase2 > 0
z_phase2 = z_phase2[mask]
w_phase2 = w_phase2[mask]
t_phase2 = t_phase2[mask]

5. Plotten der Ergebnisse und Interpretation

In [ ]:
## SOLUTION

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(t_phase1, z_phase1, label='phase 1')
ax1.plot(t_phase2, z_phase2, label='phase 2')
ax1.set_xlabel(r'$t$ [s]')
ax1.set_ylabel(r'$z(t)$ [m]')
ax1.legend()
ax2.plot(t_phase1, w_phase1, label='phase 1')
ax2.plot(t_phase2, w_phase2, label='phase 2')
ax2.set_xlabel(r'$t$ [s]')
ax2.set_ylabel(r'$w(t)=\dot{z}(t)$ [m/s]')
ax2.legend()

# Damit alle Achsenbeschriftungen gut passen und die
# Figuren nicht zu eng beieinander stehen.
fig.tight_layout()

In [ ]:
## SOLUTION

# alternative Lösung: zusammenfügen der beiden Lösungen
# zu einem Array
z_all = np.concatenate((z_phase1, z_phase2))
w_all = np.concatenate((w_phase1, w_phase2))
t_all = np.concatenate((t_phase1, t_phase2))

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(t_all, z_all)
ax1.set_xlabel(r'$t$ [s]')
ax1.set_ylabel(r'$z(t)$ [m]')
ax2.plot(t_all, w_all)
ax2.set_xlabel(r'$t$ [s]')
ax2.set_ylabel(r'$w(t)=\dot{z}(t)$ [m/s]')

# Damit alle Achsenbeschriftungen gut passen und die
# Figuren nicht zu eng beieinander stehen.
fig.tight_layout()

Bestimme das Maximum der Geschwindigkeit $w_{\rm max}(t)=|\dot{z}_{\rm max}(t)|$ und die Zeit ihres Auftretens $t_{\rm max}$. Erreicht Felix in unserem Modell die Schallgeschwindigkeit von 343 m/s?

In [ ]:
## SOLUTION

# Ohne Kommentare eine der Lösungsmöglichkeiten. 
# Fragen Sie bei Unklarheiten bitte in den YouTube-Kommentaren
# Die Schallgeschwindigkeit von 343 m/s wird knapp verfehlt!
mask = w_all[1:] - w_all[:-1] > 0
t_min = t_all[1:][mask][0]
w_min = w_all[1:][mask][0]
t_min, w_min

## Quellen

- [10 Jahre Stratosphärensprung auf der Red Bull Webseite](https://www.redbull.com/int-en/felix-baumgartner-stratos-10-years) 
- Nähere Erläuterungen zur Physik des Fallschirmsprungs finden sich 
[hier](https://ethz.ch/content/dam/ethz/special-interest/dual/educeth-dam/documents/Unterrichtsmaterialien/mathematik/gruene-berichte/fallschirmspringer.pdf).
- Bildquellen:
  - [Bild vor dem Absprung](https://www.redbull.com/de-de/flieg-mit-baumgartner-red-bull-stratos-pov-video)
  - [Bild vom Freien Fall](https://www.swr.de/wissen/1000-antworten/wie-schnell-kann-ein-mensch-vom-himmel-fallen-100.html)
  - [Bild mit geöffnetem Fallschirm](https://www.faz.net/aktuell/gesellschaft/menschen/sprung-aus-36-500-metern-am-fallschirm-zwischen-schoenheit-und-tod-11909348.html)
  - [Bild von der Landung](https://www.spiegel.de/wissenschaft/weltall/felix-baumgartner-springt-aus-39-kilometern-zur-erde-a-861241.html)
  - [Logo Fallschirmsprung 1](https://de.vecteezy.com/vektorkunst/17127790-fallschirm-logo-icon-design-und-symbol-fallschirmsprung-vektor)
  - [Logo Fallschirmsprung 2](https://stock.adobe.com/de/search?k=skydive+logo)

## Anhang

### Diiferentialgleichungen höherer Ordnung

Das Verfahren zur Überführung von Gleichungen zweiter Ordnung in ein gekoppeltes System von DGL erster Ordnung kann in natürlicher Weise auf höhere Ordnungen fortgesetzt werden. Wir wollen den allgemeinen Fall anhand eines Systems 3. Ordnung exemplarisch behandeln.

**Beispiel:** Betrachte folgendes System aus DGL 3. Ordnung: 

\begin{eqnarray}
\dddot{x_1} & = & -2\dot{x_2}^2 + x_2 \tag{2} \\
\dddot{x_2} & = & -\ddot{x_1}^3 + \dot{x_2} + x_1 + \sin(t) \tag{3}
\end{eqnarray}

Hier müssen neue Variable für `alle` Ableitungen ausschliesslich der höchsten Ordnung definiert werden.

Definiere für unseren Fall:

- $v_1 = \dot{x_1}$
- $v_2 = \dot{x_2}$
- $w_1 = \ddot{x_1} = \dot{v_1}$
- $w_2 = \ddot{x_2} = \dot{v_2}$

Dies sind 4 gekoppelte DGL erster Ordnung! Mit $\dot{w_1} = \dddot{x_1}$ und $\dot{w_2} = \dddot{x_2}$ ergibt sich für die ursprünglichen Gleichungen (2) und (3):

- $\dot{w_2} = -w_1^3 + v_2 + x_1 + \sin(t)$
- $\dot{w_1} = -2v_2^2 + x_2$

Und weiter mit unserem Lösungsschema:

$$\vec{S} = \begin{bmatrix} x_1\\ x_2 \\ v_1 \\ v_2 \\ w_1 \\ w_2 \end{bmatrix} \hspace{10mm} \implies \hspace{10mm} \frac{d\vec{S}}{dt} = \begin{bmatrix} \dot{x_1}\\ \dot{x_2} \\ \dot{v_1} \\ \dot{v_2} \\ \dot{w_1} \\ \dot{w_2} \end{bmatrix} =  \begin{bmatrix} v_1\\ v_2 \\ w_1 \\ w_2 \\ -2v_2^2 + x_2 \\ -w_1^3 + v_2 + x_1 + \sin(t) \end{bmatrix}$$

In [ ]:
def dSdt(S, t):
    x1, x2, v1, v2, w1, w2 = S
    return [v1,
            v2,
            w1,
            w2,
            -2 * v2**2 + x2,
            -w1**3 + v2 + v1 + np.sin(t)]

In [ ]:
t = np.linspace(0, 1.0, 100)
x1_0 = 0
x2_0 = 0
v1_0 = 0
v2_0 = 0
w1_0 = 0
w2_0 = 0
S_0 = (x1_0, x2_0, v1_0, v2_0, w1_0, w2_0)

S = odeint(dSdt, t=t, y0=S_0).T

x1_sol, x2_sol, v1_sol, v2_sol, w1_sol, w2_sol = S

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

ax1.plot(t, x1_sol, label="x1_sol")
ax1.set_xlabel('t')
ax1.set_ylabel('x1')
ax2.plot(t, x2_sol, label="x2_sol")
ax2.set_xlabel('t')
ax2.set_ylabel('x2')
ax3.plot(t, w1_sol, label="w1_sol")
ax3.set_xlabel('t')
ax3.set_ylabel('w1')

ax1.legend()
ax2.legend()
ax3.legend()

fig.tight_layout()